## Ingest Races.csv file

In [0]:
%fs
ls /mnt/prisha12/raw/

### 1. reading csv file, trying to fiugre out how is the format

In [0]:
#defining schema for data
from pyspark.sql.types import IntegerType, StringType,StructField,StructType

race_schema = StructType([StructField("raceId",IntegerType(),False),
                          StructField("year",IntegerType(),True),
                          StructField("round",IntegerType(),True),
                          StructField("circuitId",IntegerType(),True),
                          StructField("name",StringType(),True),
                          StructField("date",StringType(),True),
                          StructField("time",StringType(),True),
                          StructField("url",StringType(),True)])
                           

In [0]:
df = spark.read.csv('dbfs:/mnt/prisha12/raw/races.csv',header=True,schema=race_schema)
display(df.limit(10))

In [0]:
display(df.describe())

In [0]:
display(df.printSchema())

In [0]:
df_selected=df.select("raceId","year","round","circuitId","name","date","time")
display(df_selected.show())

In [0]:
df_renamed = df_selected.withColumnRenamed("raceId","race_id").withColumnRenamed("year","race_year").withColumnRenamed("circuitId","circuit_id").withColumnRenamed("date","race_date").withColumnRenamed("time","race_time")

In [0]:
## merging date and time as racetimestamp
from pyspark.sql.functions import lit,concat,col
df_renamed= df_renamed.withColumn("race_timestamp",concat(col("race_date"),lit(" "),col("race_time")))

In [0]:
display(df_renamed)

In [0]:
#drop race_date and race_time
df_renamed = df_renamed.drop("race_date","race_time")

In [0]:
##adding ingesting timestamp as ingesting date
from pyspark.sql.functions import current_timestamp
df_final = df_renamed.withColumn("ingesting_timestamp",current_timestamp())

In [0]:
display(df_final.limit(10))

In [0]:
display(df_final.printSchema())

### partitoning data by race year, it will help me optiomizing query to pull and analyze data

In [0]:
##write this data into data lake processed
df_final.write.mode("overwrite").partitionBy('race_year').format("parquet").save("/mnt/prisha12/processed/races")